All libraries imported for use through out the project

In [56]:
import os
import glob
import pandas as pd
import numpy as np
import cv2 
import math
import xml.etree.ElementTree as ET
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage.feature import hog
import matplotlib.pyplot  as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import random 
from PIL import Image
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


All file paths are created and assigned to variables for ease of use

In [57]:
pro_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\ProcessedImages"
pro_folder = os.path.realpath(pro_folder)
raw_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\RawImages"
raw_folder = os.path.realpath(raw_folder)
cropped_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\CroppedImages"
cropped_folder = os.path.realpath(cropped_folder)
rootdir_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations"
rootdir_folder = os.path.realpath(rootdir_folder)
annotations_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\Annotations"
annotations_folder = os.path.realpath(annotations_folder)
images_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\JPEGImages"
images_folder = os.path.realpath(images_folder)
neg_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\NegativeImages"
neg_folder = os.path.realpath(neg_folder)
original_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\OriginalData"
original_folder = os.path.realpath(original_folder)
model_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\Models"
model_folder = os.path.realpath(model_folder)
hnm_folder = r"C:\Users\scott\OneDrive\Desktop\m2cai16-tool-locations\HardNegativeMining"
hnm_folder = os.path.realpath(hnm_folder)


Initialises dataframe. All required data is taken from the annotations provided in original data set.

In [58]:
os.chdir(annotations_folder)
anno_list = []
for xml_file in glob.glob(annotations_folder + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for tool in root.findall('object'):
        value = (root.find('filename').text,#filename
                    int(root.find('size')[0].text),#image dimensions
                    int(root.find('size')[1].text),
                    tool[0].text,#class
                    int(tool[4][0].text),#box coordinates
                    int(tool[4][1].text),
                    int(tool[4][2].text),
                    int(tool[4][3].text)#box coordinates
                    )
        anno_list.append(value)


column_name = ['filename', 'img_width', 'img_height', 
                'class', 'xmin', 'ymin', 'xmax', 'ymax']


class_df = pd.DataFrame(anno_list, columns=column_name)
class_df

,filename,img_width,img_height,class,xmin,ymin,xmax,ymax
0,v01_002075.jpg,596,334,Grasper,201,171,256,191
1,v01_002250.jpg,596,334,Grasper,155,151,210,174
2,v01_002250.jpg,596,334,Grasper,423,154,487,287
3,v01_002425.jpg,596,334,Grasper,178,151,227,174
4,v01_004050.jpg,596,334,Grasper,240,79,315,142
...,...,...,...,...,...,...,...,...
3924,v10_065125.jpg,578,324,SpecimenBag,327,197,444,294
3925,v10_065150.jpg,578,324,SpecimenBag,125,134,348,303
3926,v10_066100.jpg,578,324,SpecimenBag,106,111,341,216
3927,v10_066100.jpg,578,324,Grasper,163,7,338,69


Class instances are counted and source images are varified to match expected number.

In [59]:
print(class_df["class"].value_counts())
print("\nUnique source images: " + str(class_df["filename"].nunique()))

Grasper        1422
Irrigator       485
SpecimenBag     476
Bipolar         450
Clipper         400
Scissors        388
Hook            308
Name: class, dtype: int64

Unique source images: 2811


Showing the range of tool instances between all images. 

In [60]:
df2 = class_df.pivot_table(columns=['filename'], aggfunc='size').copy()
print(df2.sort_values())

filename
v01_002075.jpg    1
v08_002025.jpg    1
v08_001850.jpg    1
v08_001150.jpg    1
v08_000975.jpg    1
                 ..
v03_109000.jpg    4
v05_060025.jpg    4
v05_060150.jpg    4
v09_045200.jpg    4
v03_108875.jpg    4
Length: 2811, dtype: int64


Augmented images provided by original data set are removed. Remaining images were later moved to: m2cai16-tool-locations\RawImages 

In [61]:
total_flip = 0
for image in glob.glob(original_folder + '/*flip.jpg'):
    total_flip = total_flip + 1
    #os.remove(image)

print(total_flip)#Gives total number of flipped images.

0


Applies OneHotEncoder() to all class labels are creates new column in class_df.

In [62]:
enc = OneHotEncoder()

enc.fit(class_df['class'].values.reshape(-1,1))
encoded_classes = np.asarray(enc.transform(class_df['class'].values.reshape(-1,1)).toarray())
encoded_classes = encoded_classes.tolist()

class_df['Enc. Values'] = encoded_classes
class_df

,filename,img_width,img_height,class,xmin,ymin,xmax,ymax,Enc. Values
0,v01_002075.jpg,596,334,Grasper,201,171,256,191,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
1,v01_002250.jpg,596,334,Grasper,155,151,210,174,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
2,v01_002250.jpg,596,334,Grasper,423,154,487,287,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
3,v01_002425.jpg,596,334,Grasper,178,151,227,174,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
4,v01_004050.jpg,596,334,Grasper,240,79,315,142,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...
3924,v10_065125.jpg,578,324,SpecimenBag,327,197,444,294,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3925,v10_065150.jpg,578,324,SpecimenBag,125,134,348,303,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3926,v10_066100.jpg,578,324,SpecimenBag,106,111,341,216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3927,v10_066100.jpg,578,324,Grasper,163,7,338,69,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"


Multi-labels created through elementwise addition on Enc. Values for rows that share the same filename in class_df.

In [63]:
original_df = pd.DataFrame(columns = ['Filename', 'Multi_label'])
filename = ""
multilabel = []
label_list = []
filename_list = []
for index,row in class_df.iterrows():
    if filename == row["filename"] or index == 0:
        if len(multilabel):
            multilabel = [multilabel[0] + row['Enc. Values'][0], multilabel[1] + row['Enc. Values'][1], multilabel[2] + row['Enc. Values'][2],
                          multilabel[3] + row['Enc. Values'][3], multilabel[4] + row['Enc. Values'][4], multilabel[5] + row['Enc. Values'][5],
                          multilabel[6] + row['Enc. Values'][6]]
        else:
            multilabel = row['Enc. Values']
    else:
        filename_list.append(filename)
        label_list.append(multilabel)
        multilabel = row['Enc. Values']
    filename = row["filename"]
filename_list.append(filename)
label_list.append(multilabel)
original_df['Filename'] = filename_list
original_df['Multi_label'] = label_list
original_df

,Filename,Multi_label
0,v01_002075.jpg,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
1,v01_002250.jpg,"[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]"
2,v01_002425.jpg,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
3,v01_004050.jpg,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
4,v01_004225.jpg,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...
2806,v10_065025.jpg,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
2807,v10_065125.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
2808,v10_065150.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
2809,v10_066100.jpg,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"


In [64]:
#Collection of cropped image names for each class, later used for augmentation
graspList = []
spec_bag_list = []
bipolar_list = []
clipper_list = []
scissors_list = []
hook_list = []
irrigator_list = []

newFileList = []
img_counter = 0
filename = ""

for index,row in class_df.iterrows():
    os.chdir(raw_folder)
    img = Image.open(row["filename"])
    img_crop = img.crop((row["xmin"], row["ymin"], row["xmax"], row["ymax"]))
    if filename == row["filename"]:
        img_counter = img_counter + 1
    else:
        img_counter = 0
    filename = row["filename"]
    newfilename = filename[:-4] + str(img_counter) + ".jpg"
    os.chdir(cropped_folder)
    newFileList.append(newfilename)
    #img_crop.save(newfilename)]

    if (row["class"] == "Grasper"):
        graspList.append(newfilename)
    elif (row["class"] == "SpecimenBag"):
        spec_bag_list.append(newfilename)
    elif (row["class"] == "Bipolar"):
        bipolar_list.append(newfilename)
    elif (row["class"] == "Clipper"):
        clipper_list.append(newfilename)
    elif (row["class"] == "Scissors"):
        scissors_list.append(newfilename)
    elif (row["class"] == "Hook"):
        hook_list.append(newfilename)
    elif (row["class"] == "Irrigator"):
        irrigator_list.append(newfilename)

class_df["Cropped File"] = newFileList    
class_df

,filename,img_width,img_height,class,xmin,ymin,xmax,ymax,Enc. Values,Cropped File
0,v01_002075.jpg,596,334,Grasper,201,171,256,191,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0020750.jpg
1,v01_002250.jpg,596,334,Grasper,155,151,210,174,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0022500.jpg
2,v01_002250.jpg,596,334,Grasper,423,154,487,287,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0022501.jpg
3,v01_002425.jpg,596,334,Grasper,178,151,227,174,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0024250.jpg
4,v01_004050.jpg,596,334,Grasper,240,79,315,142,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0040500.jpg
...,...,...,...,...,...,...,...,...,...,...
3924,v10_065125.jpg,578,324,SpecimenBag,327,197,444,294,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",v10_0651250.jpg
3925,v10_065150.jpg,578,324,SpecimenBag,125,134,348,303,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",v10_0651500.jpg
3926,v10_066100.jpg,578,324,SpecimenBag,106,111,341,216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",v10_0661000.jpg
3927,v10_066100.jpg,578,324,Grasper,163,7,338,69,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v10_0661001.jpg


Varifiying the contents of class lists

In [11]:
# print(graspList)
# print(spec_bag_list)
# print(bipolar_list)
# print(clipper_list)
# print(scissors_list)
# print(hook_list)
# print(irrigator_list)

['v01_0020750.jpg', 'v01_0022500.jpg', 'v01_0022501.jpg', 'v01_0024250.jpg', 'v01_0040500.jpg', 'v01_0042250.jpg', 'v01_0044500.jpg', 'v01_0046250.jpg', 'v01_0048000.jpg', 'v01_0048001.jpg', 'v01_0071250.jpg', 'v01_0071251.jpg', 'v01_0201500.jpg', 'v01_0201501.jpg', 'v01_0251501.jpg', 'v01_0252750.jpg', 'v01_0254500.jpg', 'v01_0254501.jpg', 'v01_0256250.jpg', 'v01_0258000.jpg', 'v01_0259750.jpg', 'v01_0264001.jpg', 'v01_0264501.jpg', 'v01_0269251.jpg', 'v01_0273750.jpg', 'v01_0275501.jpg', 'v01_0275502.jpg', 'v01_0275751.jpg', 'v01_0275752.jpg', 'v01_0276000.jpg', 'v01_0276001.jpg', 'v01_0276251.jpg', 'v01_0276252.jpg', 'v01_0276501.jpg', 'v01_0276502.jpg', 'v01_0276750.jpg', 'v01_0276751.jpg', 'v01_0277001.jpg', 'v01_0277002.jpg', 'v01_0277251.jpg', 'v01_0277252.jpg', 'v01_0277500.jpg', 'v01_0277501.jpg', 'v01_0277751.jpg', 'v01_0277752.jpg', 'v01_0278001.jpg', 'v01_0278002.jpg', 'v01_0278250.jpg', 'v01_0278251.jpg', 'v01_0278501.jpg', 'v01_0278502.jpg', 'v01_0278751.jpg', 'v01_027875

The outline for a for loop used to create the augmented images.
Once the augmented images were created, the lines used to save images were emitted so that the process could remain consistent for future use.

In [65]:
'''Skip if reduced in use'''

os.chdir(cropped_folder)
print(class_df["class"].value_counts())
class_df_copy = class_df.copy()
for index, row in class_df_copy.iterrows():
    if row["class"] != "Grasper":
        cropped_img = cv2.imread(row["Cropped File"])
        flipHori = cv2.flip(cropped_img, 1)
        flipHoriFile = row["Cropped File"][:-4] + "Hor.jpg"
        flipVert = cv2.flip(cropped_img, 0)
        flipVertFile = row["Cropped File"][:-4] + "Ver.jpg"
        rotate = random.randint(0, 1)  
        flip = random.randint(0, 1)  
        if rotate == 0:
            rotate_img = cv2.rotate(cropped_img, cv2.ROTATE_90_CLOCKWISE)
        if rotate == 1:
            rotate_img = cv2.rotate(cropped_img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        rotateFile = row["Cropped File"][:-4] + "Rot.jpg"
        if os.path.exists(flipHoriFile):
            new_row = {'filename': row["filename"], 'class':row["class"],
                'Cropped File':flipHoriFile}
            class_df_copy = class_df_copy.append(new_row, ignore_index=True)

        if (os.path.exists(flipVertFile)):
            new_row = {'filename': row["filename"],  'class':row["class"],
                'Cropped File':flipVertFile}
            class_df_copy = class_df_copy.append(new_row, ignore_index=True)

        if (os.path.exists(rotateFile)):
            new_row = {'filename': row["filename"], 'class':row["class"],
                'Cropped File':rotateFile}
            class_df_copy = class_df_copy.append(new_row, ignore_index=True)


class_df = class_df_copy
class_df
print(class_df["class"].value_counts())

Grasper        1422
Irrigator       485
SpecimenBag     476
Bipolar         450
Clipper         400
Scissors        388
Hook            308
Name: class, dtype: int64
Grasper        1422
Irrigator       970
SpecimenBag     952
Bipolar         900
Clipper         800
Scissors        776
Hook            616
Name: class, dtype: int64


Function used to find negative samples within an image

In [66]:
def findNeg(box_list, im_width, im_height):
    found = False
    while found == False:
        inbox = False
        randx = random.randint(0, im_width-50)   
        randy = random.randint(0, im_height-50) 
        for box in box_list:
            if ((randx >= box[0] and randx <= box[1] and randy >= box[2] and randy <= box[3]) 
            or (randx + 50 >= box[0] and randx + 50 <= box[1] and randy + 50 >= box[2] and randy + 50 <= box[3])):
                inbox = True
        if inbox == False:
            box_list.append([randx, randx + 50, randy, randy + 50])
            return(randx, randy, box_list)     

For loop used to create negative samples for all original images.
NOTE: This will not run properly unless the correct directories are present

In [ ]:
'''Creates two negative (no tool presence) images per original image'''
'''DONT TOUCH'''

NEG_PATH = os.path.join(rootdir_folder,'NegativeImages')
if not os.path.exists(NEG_PATH):
    os.mkdir(NEG_PATH)
filename = ""
box_list = []

for index, row in class_df.iterrows():
    if filename == row["filename"] or index == 0:
        box_list.append([row["xmin"],row["xmax"],row["ymin"],row["ymax"]])


    else:
        xneg1, yneg1, box_list = findNeg(box_list, row["img_width"], row["img_height"])
        xneg2, yneg2, box_list = findNeg(box_list, row["img_width"], row["img_height"])
        os.chdir(raw_folder)
        img = Image.open(filename)
        img_crop1 = img.crop((xneg1, yneg1, xneg1 + 50, yneg1 + 50))
        img_crop2 = img.crop((xneg2, yneg2, xneg2 + 50, yneg2 + 50))
        os.chdir(neg_folder)
        newfile1 = filename[:-4] + "neg1.jpg"
        newfile2 = filename[:-4] + "neg2.jpg"
        # img_crop1.save(newfile1)
        # img_crop2.save(newfile2)
        box_list = [[row["xmin"], row["xmax"], row["ymin"], row["ymax"]]]    

    filename = row["filename"]
    if index == 3928:
        xneg1, yneg1, box_list = findNeg(box_list, row["img_width"], row["img_height"])
        xneg2, yneg2, box_list = findNeg(box_list, row["img_width"], row["img_height"])
        os.chdir(raw_folder)
        img = Image.open(filename)
        img_crop1 = img.crop((xneg1, yneg1, xneg1 + 50, yneg1 + 50))
        img_crop2 = img.crop((xneg2, yneg2, xneg2 + 50, yneg2 + 50))
        os.chdir(neg_folder)
        newfile1 = filename[:-4] + "neg1.jpg"
        newfile2 = filename[:-4] + "neg2.jpg"
        # img_crop1.save(newfile1)
        # img_crop2.save(newfile2)

        

Function designed to randomly reduce the size of a list. NOTE: This function does not provide reproducability so was used exclusively for one time use or when reproducability is not a factor. 

In [68]:
def getReduced(full_list, value):
    reduced_list = []
    while len(reduced_list) < value:
        randInt = random.randint(0,len(full_list)-1)
        if(full_list[randInt] not in reduced_list):
            reduced_list.append(full_list[randInt])
    give_list = reduced_list.copy()
    return give_list

getReduced applied to appropriate lists to solve class imbalance

In [71]:
# red_spec = getReduced(spec_bag_list,9)
# red_bipo = getReduced(bipolar_list,35)
# red_clip = getReduced(clipper_list,85)
# red_scis = getReduced(scissors_list,97)
# red_hook = getReduced(hook_list,177)
# graspList = getReduced(graspList, 485)
# print(red_hook)




Varifies the instances of each class

In [70]:
print(len(spec_bag_list))
print(len(bipolar_list))
print(len(clipper_list))
print(len(scissors_list))
print(len(hook_list))
print(len(graspList))

476
450
400
388
308
485


Creates the reduced data frame, used for the majority of model creation

In [ ]:
red_class_df = class_df.copy()
reduced_df = pd.DataFrame()
for index, row in red_class_df.iterrows():
    if row["class"] == "Grasper":
        if row["Cropped File"] in  graspList:
            reduced_df = reduced_df.append(row, ignore_index=True)
    if row["class"] == "SpecimenBag":
        if row["Cropped File"] in  spec_bag_list:
            reduced_df = reduced_df.append(row, ignore_index=True)
    if row["class"] == "Bipolar":
        if row["Cropped File"] in  bipolar_list:
            reduced_df = reduced_df.append(row, ignore_index=True)
    if row["class"] == "Clipper":
        if row["Cropped File"] in  clipper_list:
            reduced_df = reduced_df.append(row, ignore_index=True)
    if row["class"] == "Scissors":
        if row["Cropped File"] in  scissors_list:
            reduced_df = reduced_df.append(row, ignore_index=True)
    if row["class"] == "Hook":
        if row["Cropped File"] in  hook_list:
            reduced_df = reduced_df.append(row, ignore_index=True)
    if row["class"] == "Irrigator":
        if row["Cropped File"] in  irrigator_list:
            reduced_df = reduced_df.append(row, ignore_index=True)

print(reduced_df["class"].value_counts())
reduced_df

Adds negative samples to data frame

In [15]:
negList = os.listdir(neg_folder)
# negList = getReduced(negList,3000)
for file in (negList):
    new_row1 = {'filename': file[:-8] + ".jpg", 'img_width':50, 'img_height':50, 'class':'Negative',
                'Cropped File':file}
    reduced_df = reduced_df.append(new_row1, ignore_index=True)
    

Adds hard negatives to data frame

In [ ]:

hnm_list = os.listdir(hnm_folder)
hnm_list = getReduced(hnm_list, 4000)
for file in (hnm_list):
    new_row1 = {'filename': file[:-9] + ".jpg", 'class':'Negative',
                'Cropped File':file}
    reduced_df = reduced_df.append(new_row1, ignore_index=True)

Varifies the class count for the data frame

In [73]:
print(reduced_df["class"].value_counts())
reduced_df

Grasper        485
Irrigator      485
SpecimenBag    476
Bipolar        450
Clipper        400
Scissors       388
Hook           308
Name: class, dtype: int64


,filename,img_width,img_height,class,xmin,ymin,xmax,ymax,Enc. Values,Cropped File
0,v01_002075.jpg,596.0,334.0,Grasper,201.0,171.0,256.0,191.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0020750.jpg
1,v01_004225.jpg,596.0,334.0,Grasper,344.0,7.0,432.0,100.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0042250.jpg
2,v01_004800.jpg,596.0,334.0,Grasper,215.0,189.0,367.0,250.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0048000.jpg
3,v01_004800.jpg,596.0,334.0,Grasper,261.0,88.0,333.0,142.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v01_0048001.jpg
4,v01_017125.jpg,596.0,334.0,Hook,190.0,129.0,258.0,187.0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",v01_0171250.jpg
...,...,...,...,...,...,...,...,...,...,...
2987,v10_065025.jpg,578.0,324.0,Grasper,125.0,70.0,329.0,137.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",v10_0650251.jpg
2988,v10_065125.jpg,578.0,324.0,SpecimenBag,327.0,197.0,444.0,294.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",v10_0651250.jpg
2989,v10_065150.jpg,578.0,324.0,SpecimenBag,125.0,134.0,348.0,303.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",v10_0651500.jpg
2990,v10_066100.jpg,578.0,324.0,SpecimenBag,106.0,111.0,341.0,216.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",v10_0661000.jpg


Emits multi-class label with only one instance

In [74]:
original_df = original_df[original_df.Filename !=  "v01_035325.jpg"]
print(original_df["Multi_label"].value_counts())

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]    385
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]    307
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]    299
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]    264
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]    245
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]    221
[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]    205
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]    185
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]    175
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]    101
[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]    100
[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0]     96
[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]     79
[0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0]     63
[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]     61
[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0]     20
[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0]      4
Name: Multi_label, dtype: int64


Train test split for the filenames assosiated with multi-labels

In [75]:
'''Train-test split on original images.'''

X_train, X_test, y_train, y_test = train_test_split((np.asarray(original_df["Filename"])), (np.asarray(original_df["Multi_label"])), test_size = 0.2,random_state=22, shuffle=True, stratify=(np.asarray(original_df["Multi_label"])))
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2248,)
(2248,)
(562,)
(562,)


Creates temporary data frames to show the successful splitting of multi-label classes

In [76]:
'''Shows split of multi-label data between train and test data'''

class_train_df = pd.DataFrame(columns = ['Filename', 'Multi-label'])
class_train_df["Filename"] = X_train
class_train_df["Multi-label"] = y_train
class_test_df = pd.DataFrame(columns = ['Filename', 'Multi-label'])
class_test_df["Filename"] = X_test
class_test_df["Multi-label"] = y_test
print(class_train_df["Multi-label"].value_counts())
print(class_test_df["Multi-label"].value_counts())

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]    308
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]    246
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]    239
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]    211
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]    196
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]    177
[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]    164
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]    148
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]    140
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]     81
[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]     80
[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0]     77
[0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]     63
[0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0]     50
[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]     49
[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0]     16
[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0]      3
Name: Multi-label, dtype: int64
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]    77
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]    61
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]    60
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]    53
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]    49
[0.0, 0.0, 1.0, 0.0, 0.0, 0

Extracts train and test files from reduced data frame (data to input into models)

In [31]:
'''Extracts rows for train dataset and validates that classes are correctly represented.'''

train_df = reduced_df.copy()
test_df = reduced_df.copy()
rows = []
for index, row in train_df.iterrows():
    if row["filename"] not in X_train:
        train_df.drop(index, inplace=True) 
    else:
        test_df.drop(index, inplace=True)
train_df = train_df.reset_index()
test_df = test_df.reset_index()
print(train_df["class"].value_counts())
print(test_df["class"].value_counts()) 

Negative       8496
Irrigator       388
SpecimenBag     381
Grasper         380
Bipolar         360
Clipper         320
Scissors        310
Hook            246
Name: class, dtype: int64
Negative       1126
Grasper         105
Irrigator        97
SpecimenBag      95
Bipolar          90
Clipper          80
Scissors         78
Hook             62
Name: class, dtype: int64


HOG transformation function created to keep consistency with parameters

In [32]:
'''Function to find HOG feature descriptors'''

def hogTransform(img):
    hog_img= hog(
    img, pixels_per_cell=(5,5), 
    cells_per_block=(2, 2), 
    orientations=9, 
    block_norm='L2-Hys', 
    transform_sqrt=True)
    return (hog_img)


All images in the training set are read in and feature descriptors are calculated
Feature descriptors are added to train_df

In [ ]:
'''Images are read in, grey-scaled, resized and hog-transformed.'''
'''A 1D feature descriptor is added to the dataframe for each image.'''

data = []
os.chdir(cropped_folder)
for index, row in train_df.iterrows():
    if row["Cropped File"].endswith("neg1.jpg") or row["Cropped File"].endswith("neg2.jpg"):
        os.chdir(neg_folder)
    elif row["Cropped File"][-9:-6] == "HNM":
        os.chdir(hnm_folder)
    else:
        os.chdir(cropped_folder)
    img = cv2.imread(row["Cropped File"], 0)
    img = resize(img, (50,50))
    hog_img = hogTransform(img)
    data.append(hog_img)

train_df["Data"] = data
print(train_df)

SVM is imported for use 
Data is put in the required format for the classifer

In [34]:
'''SVM is imported.'''
'''Data is formatted correctly for model training. This includes the use of OneHotEncoder to represent the classes.'''

from sklearn import svm

enc.fit(train_df['class'].values.reshape(-1,1))

X_data = list(np.asarray(train_df['Data']))
Y_data = np.asarray(enc.transform(train_df['class'].values.reshape(-1,1)).toarray())
Y_train = np.argmax(Y_data, axis=1)

Model is trained on the training set

In [ ]:
'''SVM model is trained.'''

clf = svm.SVC(probability=True, C=10, gamma = 0.01)
clf.fit(X_data, Y_train)

Pickle is imported
Models can be saved using the lines below

In [286]:
import pickle
os.chdir(model_folder)
pickle.dump(clf, open('SVMProto.pkl', 'wb'))

Models can be loaded in with the following command

In [63]:
clf = pickle.load(open('bigSVM.pkl', 'rb'))

The set up to find the best parameters for the SVM model

In [ ]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

grid = GridSearchCV(svm.SVC(probability=True),param_grid,refit=True,verbose=2, n_jobs=-1)
grid.fit(X_data, Y_train)

print(grid.best_estimator_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
SVC(C=10, gamma=0.01, probability=True)


Images in the test set are read in and feature descriptors are calculated

In [ ]:
'''Test images are processed (same as training data).'''

test_data = []
os.chdir(cropped_folder)
for index, row in test_df.iterrows():
    if row["Cropped File"].endswith("neg1.jpg") or row["Cropped File"].endswith("neg2.jpg"):
        os.chdir(neg_folder)
    else:
        os.chdir(cropped_folder)
    img = cv2.imread(row["Cropped File"], 0)
    img = resize(img, (50,50))
    hog_img = hogTransform(img)
    test_data.append(hog_img)

test_df["Data"] = test_data
test_df

Further varification of the train test split before a model is tested

In [228]:
print(test_df["class"].value_counts())
print(train_df["class"].value_counts())

Negative       1126
Irrigator        97
SpecimenBag      95
Bipolar          90
Grasper          89
Clipper          80
Scissors         78
Hook             62
Name: class, dtype: int64
Negative       4496
Grasper         396
Irrigator       388
SpecimenBag     381
Bipolar         360
Clipper         320
Scissors        310
Hook            246
Name: class, dtype: int64


Test data is correctly formatted for the model 

In [65]:
'''Test data is correctly formatted.'''
'''Model predicts targets for test data and root mean squared error is calculated.'''

enc.fit(test_df['class'].values.reshape(-1,1))

X_data_test = list(np.asarray(test_df['Data']))
Y_data_test = np.asarray(enc.transform(test_df['class'].values.reshape(-1,1)).toarray())

targets = np.argmax(Y_data_test, axis=1)

Model predicts labels for test data

In [76]:
svm_predicted = clf.predict(X_data_test)

test_SVM_mse = mean_squared_error(targets, svm_predicted)
test_SVM_rsme=np.sqrt(test_SVM_mse)
print(test_SVM_rsme)

1.3394340322248552


Confusion matrix and classification report is displayed

In [77]:
'''Confusion matrix for multi-class classifier is presented along with the classification report for the model.'''

label_names = [0,1,2,3,4,5,6]

cmx_SVM = confusion_matrix(targets, svm_predicted, labels=label_names)
df_SVM = pd.DataFrame(cmx_SVM, columns=label_names, index=label_names)
df_SVM.columns.name = 'prediction'
df_SVM.index.name = 'label'
print(df_SVM)
print(classification_report(targets, svm_predicted))

prediction   0   1   2   3   4     5   6
label                                   
0           56   2   2   2   6    11   7
1            3  60   0   4   3     1   4
2            6   3  60   1   0    17   8
3            1   2   2  45   4     2   3
4            5   4   2   1  76     4   2
5            3   3   3   0   1  1112   4
6            9   8   1   1   5     6  46
              precision    recall  f1-score   support

           0       0.64      0.62      0.63        90
           1       0.70      0.75      0.72        80
           2       0.83      0.62      0.71        96
           3       0.78      0.73      0.75        62
           4       0.76      0.78      0.77        97
           5       0.96      0.99      0.97      1126
           6       0.59      0.59      0.59        78
           7       0.79      0.71      0.74        95

    accuracy                           0.88      1724
   macro avg       0.76      0.72      0.74      1724
weighted avg       0.88      0.88  

Sliding window function
The image (2D array), window dimensions and step increment dimensions are given.
The function returns a list of tuples containing: the coordinates of the window and the cropped image (the image at each window).

In [36]:
def slidingWindow(image, window_width, window_height, x_step_size, y_step_size):
    img_list = []
    img_width =  image.shape[1]
    img_height = image.shape[0]
    ip_row = math.ceil((img_width - window_width) / x_step_size) + 1 #Iterations per row (Plus one accounts for range starting from 0, not 1)
    ip_column = math.ceil((img_height - window_height) / y_step_size) + 1 #Iterations per column (Plus one accounts for range starting from 0, not 1)
    for y in range(ip_column):
        for x in range(ip_row):
            x_win_pos = x*x_step_size
            y_win_pos = y*y_step_size
            if (x_win_pos + window_width > img_width):
                x_win_pos = img_width - window_width
            if (y_win_pos + window_height > img_height):
                y_win_pos = img_height - window_height
            img_crop = image[ y_win_pos:(y_win_pos + window_height),x_win_pos : (x_win_pos + window_width)]
            img_list.append(((x_win_pos, y_win_pos, x_win_pos + window_width, y_win_pos + window_height), img_crop))
    return(img_list)

In [ ]:
cv2.destroyAllWindows()

Image pyramid 
An image, scale factor and iterations are given.
The function returns a list of images in descending order of size.

In [37]:
def imagePyramid(image, scale_factor, iterations):
    imgpyr_list = []
    for i in range(iterations):
        div_val = pow(scale_factor,i)
        new_image = resize(image, (image.shape[0]//div_val,image.shape[1]//div_val))
        imgpyr_list.append((div_val,new_image))
    return(imgpyr_list)    

Function to predict boxes on a given image
Implements both sliding window and image pyramid
returns a list of tuples containing: the box for the source image, class and probability for each box

In [53]:
def findBoxes(filename, window_width, window_height):
    os.chdir(raw_folder)
    image = cv2.imread(filename, 0)
    pred_boxes = []
    for pyr_tpl in imagePyramid(image,1.4,6):
        pyr_img = pyr_tpl[1]
        pyr_scale = pyr_tpl[0]
        print(pyr_scale)
        for window_tpl in slidingWindow(pyr_img,window_width,window_height,10,10):
            prob = 0
            win_img = window_tpl[1]
            win_box = window_tpl[0]
            img = resize(win_img, (50,50))
            hog_img = hogTransform(img)
            prob_list = clf.predict_proba((hog_img).reshape(1, -1))[0]
            for index in range(len(prob_list)):
                if prob < prob_list[index]:
                    class_type = index
                    prob = prob_list[index]
            if prob >0.85 and class_type!=5 and class_type != 7:
                print((prob,class_type)) 

                pred_boxes.append(((math.ceil(win_box[0]*pyr_scale), math.ceil(win_box[1]*pyr_scale), math.ceil(win_box[2]*pyr_scale), math.ceil(win_box[3]*pyr_scale)),class_type,prob))
            
    return(pred_boxes)

function to calculate the intersection over union for two boxes

In [41]:
def IoU (box1, box2):
    #Calculates area of given boxes
    box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    #Calculates corners of intersecting box
    xtopL = max(box1[0], box2[0])
    ytopL = max(box1[1], box2[1])
    xbotR = min(box1[2], box2[2])
    ybotR = min(box1[3], box2[3])

    #Calculates area of intersecting box (gives 0 if intersecting box doesn't exist)
    interArea = max(0, xbotR - xtopL + 1) * max(0, ybotR - ytopL + 1)

    #Calculates intersection over union
    iou = interArea / float(box1Area + box2Area - interArea)
    
    return iou

Non maximum suppression function for a list of boxes

In [42]:
def nms(boxes):
    box_list = boxes.copy()
    box_list.sort(key = lambda i:i[2], reverse = True)
    new_list = []
    while len(box_list) != 0:
        new_box = box_list[0]
        new_list.append(box_list[0])
        box_list.remove(box_list[0])
        box_list_copy = box_list.copy()
        for box in box_list:
            if (IoU(new_box[0],box[0])) > 0:
                box_list_copy.remove(box)
        box_list = box_list_copy
    return new_list 

function to draw in final predicted boxes

In [27]:
def drawBox(image, box,colour, name):
    if name == "0":
        class_name = "Bipolar"
        colour = (203,131,21)
    elif name == "1":
        class_name = "Clipper"
        colour = (203,21,40)
    elif name == "2":
        class_name = "Grasper"
        colour = (214,235,25)
    elif name == "3":
        class_name = "Hook"
        colour = (25,235,228)
    elif name == "4":
        class_name = "Irrigator"
        colour = (25,116,235)
    elif name == "6":
        class_name = "Scissors"
        colour = (102,25,235)
    elif name == "7":
        class_name = "Specimen Bag"
        colour = (235,25,235)
    else:
        class_name = name
    clone = image.copy()
    clone = cv2.rectangle(clone, (box[0], box[1]), (box[2], box[3]), colour, 2)
    clone = cv2.putText(clone, class_name, (box[0], box[1]-10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2)
    return clone
    

Function to save images for the hard negative mining process

In [28]:
def saveImg(filename, box, index,folder):
    os.chdir(raw_folder)
    if index < 10:
        str_index = "0" + str(index)
    elif index > 99:
        print("EXCEEDED RANGE")    
    else:
        str_index = str(index)    
    img = Image.open(filename)
    img_crop = img.crop((box[0], box[1], box[2], box[3]))
    
    newfile = filename[:-4] + "HNM" + str_index + ".jpg"
    os.chdir(folder)
    img_crop.save(newfile)
    

function to find truth boxes for a given source image 

In [43]:
def findTruths(filename):
    truth_boxes = []
    for index, row in class_df.iterrows():
        if row["filename"] == filename and row["class"] != "Negative" and not pd.isnull(row["xmin"]):
            truth_boxes.append(((int(row["xmin"]),int(row["ymin"]),int(row["xmax"]),int(row["ymax"])),row["class"]))
    return truth_boxes

Example truth box finding

In [45]:
os.chdir(raw_folder)
test_image = "v02_027150.jpg"
truth_boxes = findTruths(test_image)
print(truth_boxes)

[((287, 207, 520, 422), 'Scissors')]


Shows filenames for test set

In [ ]:
test_imgs = []

for index, row in test_df.iterrows():
    test_imgs.append(row["filename"])


test_imgs = list(dict.fromkeys(test_imgs)) 
print(test_imgs)    

Shows filenames for training set

In [ ]:
train_imgs = []

for index, row in train_df.iterrows():
    train_imgs.append(row["filename"])
train_imgs = list(dict.fromkeys(train_imgs))    
print(train_imgs)    

Hard negative mining process

In [ ]:
counter = 0
os.chdir(raw_folder)
for hnm_image in X_test:
    counter = counter + 1
    print("IMAGE:" + str(counter) + "/" + str(len(X_test)))
    index = 0
    truth_boxes = findTruths(hnm_image)
    pred_boxes_sqr = findBoxes(hnm_image, 50, 50)
    pred_boxes_rect = findBoxes(hnm_image, 50, 25)
    concat_boxes = pred_boxes_rect + pred_boxes_sqr

    
    final_boxes = nms(concat_boxes)
    image = cv2.imread(hnm_image)
    clone2 = image.copy()
    truth_boxes = findTruths(hnm_image)

    for box_tup in final_boxes:
        clone2 = drawBox(clone2, box_tup[0], (0,150,0), str(box_tup[1]))
    for truth_tup in truth_boxes:
        print(truth_tup[0])
        clone2 = drawBox(clone2, truth_tup[0], (0,0,0), truth_tup[1])    
    cv2.imshow("Box Img", clone2)
    cv2.waitKey(0)
    #cv2.imwrite("PRED"+hnm_image, clone2)
    cv2.waitKey(0) 

Box predictions for a given test image

In [ ]:
import pickle
os.chdir(model_folder)
clf = pickle.load(open('reducedSVMv2.pkl', 'rb'))

test_image = "v02_027150.jpg"
truth_boxes = findTruths(test_image)

pred_boxes_sqr = findBoxes(test_image, 50, 50)
pred_boxes = findBoxes(test_image, 50, 25)

concat_boxes = pred_boxes + pred_boxes_sqr

Total box predictions from test

In [ ]:
print(concat_boxes)

[((330, 10, 50, 25), 2, 0.93654776059604), ((360, 100, 50, 25), 2, 0.977850000722603), ((370, 100, 50, 25), 2, 0.9677474000509877), ((280, 130, 50, 25), 4, 0.9548428908865284), ((280, 140, 50, 25), 4, 0.9258880849374127), ((360, 220, 50, 25), 4, 0.9105308563982726), ((165, 264, 138, 69), 4, 0.9254022939107489)]


Creates image for test 

In [ ]:
final_boxes = nms(concat_boxes)
image = cv2.imread(test_image)
clone2 = image.copy()
for box_tup in final_boxes:
    clone2 = drawBox(clone2, box_tup[0], (0,150,0), str(box_tup[1]))
for truth_tup in truth_boxes:
    print(truth_tup[0])
    clone2 = drawBox(clone2, truth_tup[0], (0,0,0), truth_tup[1])    
cv2.imshow("Box Img", clone2)
cv2.waitKey(0)

Saves final image if required

In [429]:
os.chdir(raw_folder)
cv2.imwrite("PREDv02_027150.jpg", clone2)
cv2.waitKey(0)

-1